In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

import joblib
import os
import logging

from datetime import datetime
import json
import yaml
from pathlib import Path
import time





# Import Data Files
affinity_df = pd.read_csv('data/Affinity - State - Daily.csv')
employment_df = pd.read_csv('data/Employment - State - Weekly.csv')


#Affinity Data Cleaing

affinity_df['date'] = pd.to_datetime(affinity_df[['year','month','day']]) # type: ignore



affinity_df['week_end'] = affinity_df['date'].dt.to_period('W-SUN').dt.end_time.dt.normalize()
aff_weekly = affinity_df.groupby(['statefips', 'week_end']).mean(numeric_only=True).reset_index()

# Employment Data Cleaning

for c in ['year', 'month', 'day_endofweek']:
    employment_df[c] = employment_df[c].astype('int64')
employment_df['date'] = pd.to_datetime( {'year': employment_df['year'], 'month': employment_df['month'], 'day': employment_df['day_endofweek']} )



#Merging

merged = employment_df.merge(affinity_df, on=['statefips', 'date'], how='left')



print(merged['spend_all'].dtype)

merged['spend_all'] = merged['spend_all'].astype(float)

#Creating Lag Features

merged['spend_all_lag_1'] = merged.groupby('statefips')['spend_all'].shift(1)
merged['spend_all_lag_3'] = merged.groupby('statefips')['spend_all'].shift(3)
merged['emp_next_week']   = merged.groupby('statefips')['emp'].shift(-1)

#Train/Test Split

train = merged[merged['date'] <= '2023-12-31']
test = merged[merged['date'] > '2024-01-01']

# X and Y Split (X is Spending, Y is Employment)
X = merged[['spend_all', 'spend_all_lag_1', 'spend_all_lag_3']]
y = merged['emp_next_week']


X_train = train[['spend_all', 'spend_all_lag_1', 'spend_all_lag_3']]

y_train = train['emp_next_week']

X_test  = test[['spend_all', 'spend_all_lag_1', 'spend_all_lag_3']]
y_test  = test['emp_next_week']

# Training Model


#df = pd.DataFrame({'col': ['1.0', '2.5', '3']})
#df['col'] = df['col'].astype(float)
#print(df.dtypes)

X

#model = LinearRegression()
#model.fit(X_train, y_train)
#preds = model.predict(X_test)
#mae = mean_absolute_error(y_test, preds)





object


ValueError: could not convert string to float: '.'